# Build a Chatbot

https://python.langchain.com/docs/tutorials/chatbot/
old version: https://python.langchain.com/v0.2/docs/tutorials/chatbot/




## Overview


We'll go over an example of how to design and implement an LLM-powered chatbot.   
This chatbot will be able to have a conversation and remember previous interactions with a chat model.

https://python.langchain.com/docs/tutorials/chatbot/



Note that this chatbot that we build will only use the language model to have a conversation.   
There are several other related concepts that you may be looking for:

Conversational RAG: Enable a chatbot experience over an external source of data  
Agents: Build a chatbot that can take actions  


https://python.langchain.com/docs/tutorials/qa_chat_history/
https://python.langchain.com/docs/tutorials/agents/

This tutorial will cover the basics which will be helpful for those two more advanced topics, but feel free to skip directly to there should you choose.


In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("LANGSMITH_API_KEY"):
  os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for LANGSMITH: ")


## Quickstart

First up, let's learn how to use a language model by itself.   
LangChain supports many different language models that you can use interchangeably - select the one you want to use below!

In [2]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

Let's first use the model directly.   
ChatModels are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them.   
To just simply call the model, we can pass in a list of messages to the .invoke method.




In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好! 我是 tiankonguse")])

AIMessage(content='你好，tiankonguse！很高兴和你交流。有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 15, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-fc623887-205e-4790-99ed-0cb1a9bf9848-0', usage_metadata={'input_tokens': 15, 'output_tokens': 22, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

The model on its own does not have any concept of state. For example, if you ask a followup question:

In [5]:
model.invoke([HumanMessage(content="我叫什么名字?")])

AIMessage(content='抱歉，我无法知道您的名字。不过，您可以告诉我您叫什么名字，或者有其他问题想问我吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 11, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-201eeef6-2b47-4c4a-acd4-6aac2214b769-0', usage_metadata={'input_tokens': 11, 'output_tokens': 27, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

To get around this, we need to pass the entire conversation history into the model. Let's see what happens when we do that:  
https://python.langchain.com/docs/concepts/chat_history/  

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="你好! 我是 tiankonguse"),
        AIMessage(content="你好，tiankonguse！很高兴和你交流。有什么我可以帮助你的吗？"),
        HumanMessage(content="我叫什么名字?"),
    ]
)

AIMessage(content='你刚才提到你的名字是 tiankonguse。有什么我可以为你做的呢？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 48, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-b5317c6a-c715-429b-86b0-678c0f585f7b-0', usage_metadata={'input_tokens': 48, 'output_tokens': 23, 'total_tokens': 71, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

And now we can see that we get a good response!  

This is the basic idea underpinning a chatbot's ability to interact conversationally.   
So how do we best implement this?  

## Message persistence


LangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.  
https://langchain-ai.github.io/langgraph/

Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.


LangGraph comes with a simple in-memory checkpointer, which we use below.   
See its documentation for more detail, including how to use different persistence backends (e.g., SQLite or Postgres).

https://langchain-ai.github.io/langgraph/concepts/persistence/


In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

We now need to create a config that we pass into the runnable every time.   


This config contains information that is not part of the input directly, but is still useful.   

In this case, we want to include a thread_id. This should look like:

In [8]:
config = {"configurable": {"thread_id": "abc123"}}



This enables us to support multiple conversation threads with a single application, a common requirement when your application has multiple users.

We can then invoke the application:


In [ ]:
query = "你好! 我是 tiankonguse."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

# ================================== Ai Message ==================================
#
# 你好，tiankonguse！很高兴和你交流。有什么我可以帮助你的吗？

================================== Ai Message ==================================

你好，tiankonguse！很高兴和你交流。有什么我可以帮助你的吗？


In [ ]:
query = "我的名字是什么?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# ================================== Ai Message ==================================
#
# 你的名字是 tiankonguse。有什么特别的含义或者故事吗？

================================== Ai Message ==================================

你的名字是 tiankonguse。有什么特别的含义或者故事吗？


Great!   
Our chatbot now remembers things about us.   
If we change the config to reference a different thread_id, we can see that it starts the conversation fresh.

In [ ]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# ================================== Ai Message ==================================
#
# 抱歉，我无法知道你的名字。如果你愿意，可以告诉我你的名字，或者问我其他问题！

================================== Ai Message ==================================

抱歉，我无法知道你的名字。如果你愿意，可以告诉我你的名字，或者问我其他问题！


However, we can always go back to the original conversation (since we are persisting it in a database)



In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# ================================== Ai Message ==================================
#
# 你的名字是 tiankonguse。如果你有其他问题或者想聊的话题，请告诉我！

================================== Ai Message ==================================

你的名字是 tiankonguse。如果你有其他问题或者想聊的话题，请告诉我！


This is how we can support a chatbot having conversations with many users!



For async support, update the call_model node to be an async function and use .ainvoke when invoking the application:

In [ ]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# ================================== Ai Message ==================================
# 
# 抱歉，我无法知道你的名字。如果你愿意，可以告诉我你的名字

================================== Ai Message ==================================

抱歉，我无法知道你的名字。如果你愿意，可以告诉我你的名字。


Right now, all we've done is add a simple persistence layer around the model.  
We can start to make the chatbot more complicated and personalized by adding in a prompt template.

## Prompt templates

Prompt Templates help to turn raw user information into a format that the LLM can work with.   
In this case, the raw user input is just a message, which we are passing to the LLM.   
Let's now make that a bit more complicated.   

First, let's add in a system message with some custom instructions (but still taking messages as input).   
Next, we'll add in more input besides just the messages.

To add in a system message, we will create a ChatPromptTemplate.   
We will utilize MessagesPlaceholder to pass all the messages in.

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你说话像海盗。尽你所能回答所有问题。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

We can now update our application to incorporate this template:



In [16]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

We invoke the application in the same way:



In [ ]:
config = {"configurable": {"thread_id": "abc345"}}
query = "你好! 我是 tiankonguse."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# ================================== Ai Message ==================================
#
# Ahoy, tiankonguse! 欢迎 aboard me ship! 你有啥问题要问这位海盗吗？无论是寻宝的方向还是航海的技巧，我都乐意分享，呔！🏴‍☠️✨

================================== Ai Message ==================================

Ahoy, tiankonguse! 欢迎 aboard me ship! 你有啥问题要问这位海盗吗？无论是寻宝的方向还是航海的技巧，我都乐意分享，呔！🏴‍☠️✨


In [ ]:
query = "我的名字是什么?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# ================================== Ai Message ==================================
# 
# Ahoy! 你自称 tiankonguse！这可是一个响亮的名字，像是来自遥远的海域、勇敢的探险者啊！你想知道些什么，或者告诉老海盗更多关于你的事情吗？🏴‍☠️⚓️

================================== Ai Message ==================================

Ahoy! 你自称 tiankonguse！这可是一个响亮的名字，像是来自遥远的海域、勇敢的探险者啊！你想知道些什么，或者告诉老海盗更多关于你的事情吗？🏴‍☠️⚓️


Awesome! Let's now make our prompt a little bit more complicated.   
Let's assume that the prompt template now looks something like this:

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个乐于助人的助手。尽你所能回答所有问题，如果不知道答案，请说 '我不知道'。请用 {language} 回答。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Note that we have added a new language input to the prompt.   
Our application now has two parameters-- the input messages and language.   
We should update our application's state to reflect this:



In [19]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc456"}}
query = "嗨，我是tiankonguse。"
language = "中文"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

# ================================== Ai Message ==================================
#
# Ahoy, tiankonguse! 欢迎登上这艘海盗船！准备好接受冒险的挑战了吗？有何问题要问我，或是想要分享的宝藏故事？

================================== Ai Message ==================================

Ahoy, tiankonguse! 欢迎登上这艘海盗船！准备好接受冒险的挑战了吗？有何问题要问我，或是想要分享的宝藏故事？


Note that the entire state is persisted, so we can omit parameters like language if no changes are desired:



In [ ]:
query = "我的名字是什么?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

# ================================== Ai Message ==================================
#
# Ahoy! 你的名字是 tiankonguse！就像一颗璀璨的星星，照亮了这片海洋！还有什么我能为你效劳的么？

================================== Ai Message ==================================

Ahoy! 你的名字是 tiankonguse！就像一颗璀璨的星星，照亮了这片海洋！还有什么我能为你效劳的么？
